In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(3,name='x')
y = tf.Variable(4,name='y')
f= x*x*y + y + 2

In [3]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


# Alternate way

In [4]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [5]:
result

42

# use global varialbe initializer to create a node in graph that will initialize all variables when run

In [6]:
init = tf.global_variables_initializer() # prepare an init node

In [7]:
with tf.Session() as sess:
    init.run()
    result=f.eval()
print(result)

42


# Runing an interactive session

In [8]:
sess = tf.InteractiveSession()
init.run()
reuslt = f.eval()
print(result)

42


# Managing Graphs 
## Any node created is added to default graph

In [11]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

# Inorder to manage multiple independed graphs do the following

In [12]:
graph=tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [13]:
x2.graph is graph

True

In [14]:
x2.graph is tf.get_default_graph()

False

In [15]:
tf.reset_default_graph()

# Life cycle 
## when you evaluate a node tensorflow automatically determines set o fnodes that it depends on and evaluates those nodes first

In [17]:
w = tf.constant(3)
x = w + 3
y = x + 5
z = x * 3
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

11
18


# note : The preceding code evaluates W and x twice , once for y.eval and once for z.eval

with tf.Session() as sess :
    y_val,z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

In [20]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [22]:
housing = fetch_california_housing()
m,n = housing.data.shape


In [23]:
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data]

In [24]:
X = tf.constant(housing_data_plus_bias,dtype=tf.float32,name='x')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

In [25]:
with tf.Session() as sess:
    theta_value = theta.eval()

In [26]:
theta_value

array([[ -3.74651413e+01],
       [  4.35734153e-01],
       [  9.33829229e-03],
       [ -1.06622010e-01],
       [  6.44106984e-01],
       [ -4.25131839e-06],
       [ -3.77322501e-03],
       [ -4.26648885e-01],
       [ -4.40514028e-01]], dtype=float32)

# Implementing gradient descent

In [31]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [34]:
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

In [47]:
n_epochs = 1000
learning_rate = 0.01

In [48]:
X = tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name='X')
y = tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name='y')
theta = tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred = tf.matmul(X,theta,name='predictions')

In [58]:
error = y_pred - y
mse = tf.reduce_mean(tf.square(error),name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X),error)
#gradients = tf.gradients(mse,[theta])[0] # using built in gradient finder (user reverse mode autodiff)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [59]:
#training_op = tf.assign(theta,theta-learning_rate*gradients)

In [60]:
init = tf.global_variables_initializer()

In [61]:
with tf.Session() as sess :
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoc',epoch,"MSE = ",mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoc 0 MSE =  8.61937
Epoc 100 MSE =  4.94447
Epoc 200 MSE =  4.86462
Epoc 300 MSE =  4.85109
Epoc 400 MSE =  4.84169
Epoc 500 MSE =  4.83428
Epoc 600 MSE =  4.82839
Epoc 700 MSE =  4.82369
Epoc 800 MSE =  4.81991
Epoc 900 MSE =  4.81687


# tensorflow placeholders

In [62]:
A = tf.placeholder(tf.float32,shape=(None,3))
B = A + 5
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2 = B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})

In [63]:
print(B_val_1)

[[ 6.  7.  8.]]
